# Preparation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
GOOGLE_DRIVE_PATH_AFTER_MYDRIVE = 'CS598_DL4H'
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive', GOOGLE_DRIVE_PATH_AFTER_MYDRIVE)
print(os.listdir(GOOGLE_DRIVE_PATH))
os.chdir(GOOGLE_DRIVE_PATH)

['mimic-iii-clinical-database-1.4', 'mimic3benchmark', 'mimic3models', 'data', 'utils', 'model', 'prepare_dataset.ipynb', 'demographic_data.txt', 'diagnosis_data.txt', 'idx_list.txt', 'demographic.ipynb', 'ConCare (1).ipynb', 'retain-master', 'Retain_Test.ipynb', 'concare-notebook.ipynb', 'ConCare.ipynb']


In [4]:
%cd mimic3-benchmarks-master

[Errno 2] No such file or directory: 'mimic3-benchmarks-master'
/content


In [5]:
import numpy as np
import argparse
import os
import imp
import re
import pickle
import datetime
import random
import math
import copy


import torch
from torch import nn
import torch.nn.utils.rnn as rnn_utils
from torch.utils.data import DataLoader, Dataset
from torch.autograd import Variable
import torch.nn.functional as F

from utils import utils
from utils.readers import InHospitalMortalityReader
from utils.preprocessing import Discretizer, Normalizer
from utils import metrics
from utils import common_utils


# Data Preprocessing

In [6]:
TIMESTEP = 1.0
data_path = 'data/hospital-mortality/'
# Build readers, discretizers, normalizers
train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                         listfile=os.path.join(data_path, 'train_listfile.csv'),
                                         period_length=48.0)

val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'train'),
                                       listfile=os.path.join(data_path, 'val_listfile.csv'),
                                       period_length=48.0)

discretizer = Discretizer(timestep=TIMESTEP,
                          store_masks=True,
                          impute_strategy='previous',
                          start_time='zero')

In [7]:
discretizer_header = discretizer.transform(train_reader.read_example(0)["X"])[1].split(',')
cont_channels = [i for (i, x) in enumerate(discretizer_header) if x.find("->") == -1]

normalizer = Normalizer(fields=cont_channels)  # choose here which columns to standardize
normalizer_state = 'ihm_normalizer'
normalizer_state = os.path.join(os.path.dirname(data_path), normalizer_state)
normalizer.load_params(normalizer_state)

In [9]:
train_raw = utils.load_data(train_reader, discretizer, normalizer, return_names=True)
val_raw = utils.load_data(val_reader, discretizer, normalizer, return_names=True)

In [10]:
demographic_data = []
with open('demographic_data.txt', 'r') as f:
  for line in f:
    demographic_data.append(np.fromstring(line, dtype=float, sep=' '))

diagnosis_data = []
with open('diagnosis_data.txt', 'r') as f:
  for line in f:
    diagnosis_data.append(np.fromstring(line, dtype=int, sep=' '))

idx_list = []
with open('idx_list.txt', 'r') as f:
  for line in f:
    idx_list.append(line.strip())

len(demographic_data)

<ipython-input-10-761523da05f3>:9: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  diagnosis_data.append(np.fromstring(line, dtype=int, sep=' '))


42125

In [11]:
class Dataset(Dataset):
  def __init__(self, x, y, name):
    self.x = x
    self.y = y
    self.name = name
  
  def __getitem__(self, index):
    return self.x[index], self.y[index], self.name[index]
  
  def __len__(self):
    return len(self.x)

In [12]:
BATCH_SIZE = 256
train_dataset = Dataset(train_raw['data'][0], train_raw['data'][1], train_raw['names'])
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = Dataset(val_raw['data'][0], val_raw['data'][1], val_raw['names'])
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ConCare Model

In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() == True else 'cpu')
print("available device: {}".format(device))

available device: cuda:0


In [14]:
class DemographicEmbed(nn.Module):
  def __init__(self, demo_input_dim, hidden_dim):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.hidden_dim = hidden_dim

    self.fc = nn.Linear(self.demo_input_dim, self.hidden_dim)
    # self.dropout = nn.Dropout(dropout)
  
  def forward(self, x):
    return F.tanh(self.fc(x))

In [15]:
class TimeAwareSelfAttention(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w_qs = nn.Parameter(torch.randn(input_dim, hidden_dim))
    self.w_ks = nn.Parameter(torch.randn(input_dim, hidden_dim))
    self.beta = nn.Parameter(torch.zeros(1)+0.8)

    nn.init.kaiming_uniform_(self.w_qs, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_ks, a=math.sqrt(5))
  
  def forward(self, input):
    b, time_step, input_dim = input.size()
    time_decays = torch.tensor(range(47, -1, -1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device) # 1*T*1
    b_time_decays = time_decays.repeat(b, 1, 1)+1 # B*T*1

    q = torch.matmul(input[:, -1, :], self.w_qs).reshape(b, 1, self.hidden_dim) # B*1*H
    k = torch.matmul(input, self.w_ks) # B*T*H
    v = input # B*T*H
    product = torch.matmul(q, k.transpose(1, 2)).squeeze() # B*T
    denominator = self.beta * torch.log(math.e + (1 - F.sigmoid(product)) * (b_time_decays.squeeze()))
    weights = F.softmax(F.tanh(product / denominator)) # B*T
    output = torch.matmul(weights.unsqueeze(1), v).squeeze() # B*H

    return output, weights

In [16]:
class ScaledDotProductAttention(nn.Module):
  def __init__(self, dropout=0.1):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    
  def forward(self, q, k, v):
    d_k = q.size(-1)
    attn = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    attn = self.dropout(F.softmax(attn, dim=-1))
    output = torch.matmul(attn, v)

    return output, attn

In [17]:
def cov(u):
  miu = torch.mean(u, dim=1).unsqueeze(1)
  diff = u - miu
  cov = torch.mm(diff, diff.T) / (diff.size(1)-1)
  return cov

def cross_head_decorr(u):
  covs = cov(u[0, :, :])
  loss = 0.5 * (torch.norm(covs, p='fro') ** 2 - torch.norm(torch.diag(covs)) ** 2)
  for i in range(u.size(0)-1):
    covs = cov(u[i+1, :, :])
    loss += 0.5 * (torch.norm(covs, p='fro') ** 2 - torch.norm(torch.diag(covs)) ** 2)
  return loss

In [18]:
class MultiHeadAttention(nn.Module):
  def __init__(self, n_head, d_model):
    super().__init__()
    self.n_head = n_head
    self.d_model = d_model

    self.w_qs = nn.Linear(self.d_model, self.d_model, bias=False)
    self.w_ks = nn.Linear(self.d_model, self.d_model, bias=False)
    self.w_vs = nn.Linear(self.d_model, self.d_model, bias=False)
    self.fc = nn.Linear(self.d_model, self.d_model, bias=False)

    self.attn = ScaledDotProductAttention()

    # self.dropout = nn.Dropout(dropout)
    # self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

  def forward(self, q, k, v):
    b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
    q = self.w_qs(q).view(b, len_q, self.n_head, self.d_model // self.n_head)
    k = self.w_ks(k).view(b, len_k, self.n_head, self.d_model // self.n_head)
    v = self.w_vs(v).view(b, len_v, self.n_head, self.d_model // self.n_head)
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2) # B*num_heads*d_input*d_k

    q, attn = self.attn(q, k, v)

    q = q.transpose(1, 2).contiguous().view(b, len_q, self.d_model) # B*d_input*d_model
    output_q = self.fc(q)

    # DeCov
    u = q.transpose(0, 1).transpose(1, 2) # d_input*d_model*B
    decov_loss = cross_head_decorr(u)

    return output_q, decov_loss


In [19]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, ff_hidden, dropout=0.1):
    super().__init__()
    self.fc1 = nn.Linear(d_model, ff_hidden)
    self.fc2 = nn.Linear(ff_hidden, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    output = self.fc2(self.dropout(F.relu(self.fc1(x))))
    return output

In [20]:
class ResConnect(nn.Module):
  def __init__(self, norm_shape, dropout=0.5):
    super().__init__()
    self.lnorm = nn.LayerNorm(norm_shape)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x, layer):
    res = layer(self.lnorm(x))
    output = x + self.dropout(res[0])
    return output, res[1]

In [21]:
class FinalAttention(nn.Module):
  def __init__(self, input_dim, hidden_dim, dropout=0.5):
    super().__init__()

    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w_qs = nn.Linear(input_dim, hidden_dim)
    self.w_ks = nn.Linear(input_dim, hidden_dim)
    self.w_vs = nn.Linear(input_dim, hidden_dim)

    # Parameter Initialization
    nn.init.kaiming_uniform_(self.w_qs.weight, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_ks.weight, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_vs.weight, a=math.sqrt(5))
    
    self.dropout = nn.Dropout(dropout)
  
  def forward(self, input):
    batch_size, time_step, input_dim = input.size()
    q = self.w_qs(input[:, -1, :]).reshape(batch_size, self.hidden_dim, 1) # B*H*1
    k = self.w_ks(input) # B*T*H
    v = self.w_vs(input) # B*T*H

    attention = F.tanh(torch.matmul(k, q).squeeze())
    score = F.softmax(attention)

    score = self.dropout(score)

    v = torch.matmul(score.unsqueeze(1), v).squeeze()

    return v, score

In [22]:
class ConCare(nn.Module):
  def __init__(self, demo_input_dim, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.demo_embed = DemographicEmbed(self.demo_input_dim, self.hidden_dim1)
    self.grus = nn.ModuleList([copy.deepcopy(nn.GRU(1, self.hidden_dim1, batch_first=True)) for _ in range(self.input_dim)])
    self.time_attns = nn.ModuleList([copy.deepcopy(TimeAwareSelfAttention(self.hidden_dim1, self.hidden_dim2)) for _ in range(self.input_dim)])
    self.multi_attn = MultiHeadAttention(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=self.dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=0.1)
    self.output_fc1 = nn.Linear(self.hidden_dim1, self.hidden_dim1)
    self.output_fc2 = nn.Linear(self.hidden_dim1, self.output_dim)
    self.dp = nn.Dropout(self.dropout)
  
  def forward(self, input, demo):
    # Demographic embedding
    demo_embedding = self.demo_embed(demo)

    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    record_embedding1 = self.grus[0](input[:, :, 0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
    time_attn_output = self.time_attns[0](record_embedding1)[0].unsqueeze(1) # B*1*H

    # All other records embeddings
    for i in range(1, feat_dim):
      embedding = self.grus[i](input[:, :, i].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
      attn = self.time_attns[i](embedding)[0].unsqueeze(1) # B*1*H
      time_attn_output = torch.cat((time_attn_output, attn), 1)
    
    # Combine with demographic embedding
    demo_embedding = demo_embedding.unsqueeze(1)
    time_attn_output = torch.cat((time_attn_output, demo_embedding), 1)
    time_attn_output = self.dp(time_attn_output)

    # Get multi-head attention
    multi_attn_output, dev_loss = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc2(F.relu(self.output_fc1(value))))

    return output, dev_loss

### Training

In [23]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

In [24]:
model = ConCare(demo_input_dim=12, input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [25]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare0'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get batch demographic data
    batch_demo = []
    for i in range(len(batch_name)):
      cur_id, cur_ep, _ = batch_name[i].split('_', 2)
      cur_idx = cur_id + '_' + cur_ep
      cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
      batch_demo.append(cur_demo)
    batch_demo = torch.stack(batch_demo).to(device)

    # Get model outputs
    output, decov_loss = model(batch_x, batch_demo)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    loss += 800 * decov_loss
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get batch demographic data
      batch_demo = []
      for i in range(len(batch_name)):
        cur_id, cur_ep, _ = batch_name[i].split('_', 2)
        cur_idx = cur_id + '_' + cur_ep
        cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
        batch_demo.append(cur_demo)
      batch_demo = torch.stack(batch_demo).to(device)

      # Get model outputs
      output, decov_loss = model(batch_x, batch_demo)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      loss += 10 * decov_loss
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 0.7203
Epoch 0 Batch 50: Train Loss = 0.4778
Epoch 0: Train Loss = 0.4703
Epoch 0: Validation Loss = 0.3991
confusion matrix:
[[2726    0]
 [ 432    0]]


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.6345500971440993
AUC of PRC = 0.2329938027896914
min(+P, Se) = 0.27253218884120173
f1_score = nan
Epoch 0: Validation AUROC = 0.6346

------------ Save best model ------------



<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

Epoch 97 Batch 0: Train Loss = 0.3272
Epoch 97 Batch 50: Train Loss = 0.2724
Epoch 97: Train Loss = 0.2715
Epoch 97: Validation Loss = 0.3047
confusion matrix:
[[2659   67]
 [ 314  118]]
accuracy = 0.8793540000915527
precision class 0 = 0.8943827748298645
precision class 1 = 0.637837827205658
recall class 0 = 0.9754218459129333
recall class 1 = 0.27314814925193787
AUC of ROC = 0.837399968750849
AUC of PRC = 0.4979733084545902
min(+P, Se) = 0.4897025171624714
f1_score = 0.3824959598197426
Epoch 97: Validation AUROC = 0.8374


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 0: Train Loss = 0.2635
Epoch 98 Batch 50: Train Loss = 0.2779
Epoch 98: Train Loss = 0.2779
Epoch 98: Validation Loss = 0.3014
confusion matrix:
[[2644   82]
 [ 302  130]]
accuracy = 0.8784040808677673
precision class 0 = 0.8974881172180176
precision class 1 = 0.6132075190544128
recall class 0 = 0.9699193239212036
recall class 1 = 0.30092594027519226
AUC of ROC = 0.8352830086138964
AUC of PRC = 0.49399254835651907
min(+P, Se) = 0.4930555555555556
f1_score = 0.4037267280569563
Epoch 98: Validation AUROC = 0.8353


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 0: Train Loss = 0.3127
Epoch 99 Batch 50: Train Loss = 0.2736
Epoch 99: Train Loss = 0.2729
Epoch 99: Validation Loss = 0.3088
confusion matrix:
[[2638   88]
 [ 307  125]]
accuracy = 0.8749208450317383
precision class 0 = 0.8957555294036865
precision class 1 = 0.5868544578552246
recall class 0 = 0.967718243598938
recall class 1 = 0.28935185074806213
AUC of ROC = 0.8358468519877176
AUC of PRC = 0.4880441170176276
min(+P, Se) = 0.48842592592592593
f1_score = 0.38759691092953846
Epoch 99: Validation AUROC = 0.8358


### Test

In [26]:
file_name = 'model/concare0'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
                                            listfile=os.path.join(data_path, 'test_listfile.csv'),
                                            period_length=48.0)
test_raw = utils.load_data(test_reader, discretizer, normalizer, return_names=True)
test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [27]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)[0]

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2805   57]
 [ 299   75]]
accuracy = 0.8899876475334167
precision class 0 = 0.9036726951599121
precision class 1 = 0.5681818127632141
recall class 0 = 0.9800838828086853
recall class 1 = 0.20053476095199585
AUC of ROC = 0.8473702993680797
AUC of PRC = 0.4350551117589894
min(+P, Se) = 0.4597402597402597
f1_score = 0.2964426887508601


# Comparison

## ConCare_MC

In [28]:
class ConCare_MC(nn.Module):
  def __init__(self, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.grus = nn.ModuleList([copy.deepcopy(nn.GRU(1, self.hidden_dim1, batch_first=True)) for _ in range(self.input_dim)])
    self.time_attns = nn.ModuleList([copy.deepcopy(TimeAwareSelfAttention(self.hidden_dim1, self.hidden_dim2)) for _ in range(self.input_dim)])
    self.multi_attn = MultiHeadAttention(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=self.dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=0.1)
    self.output_fc1 = nn.Linear(self.hidden_dim1, self.hidden_dim1)
    self.output_fc2 = nn.Linear(self.hidden_dim1, self.output_dim)
    self.dp = nn.Dropout(self.dropout)
  
  def forward(self, input):
    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    record_embedding1 = self.grus[0](input[:, :, 0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
    time_attn_output = self.time_attns[0](record_embedding1)[0].unsqueeze(1) # B*1*H

    # All other records embeddings
    for i in range(1, feat_dim):
      embedding = self.grus[i](input[:, :, i].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
      attn = self.time_attns[i](embedding)[0].unsqueeze(1) # B*1*H
      time_attn_output = torch.cat((time_attn_output, attn), 1)
    
    time_attn_output = self.dp(time_attn_output)

    # Get multi-head attention
    multi_attn_output, dev_loss = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc2(F.relu(self.output_fc1(value))))

    return output, dev_loss

### Training

In [29]:
model = ConCare_MC(input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [30]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare_mc'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get model outputs
    output, decov_loss = model(batch_x)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    loss += 800 * decov_loss
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get model outputs
      output, decov_loss = model(batch_x)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      loss += 10 * decov_loss
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 0.6937
Epoch 0 Batch 50: Train Loss = 0.4758
Epoch 0: Train Loss = 0.4675
Epoch 0: Validation Loss = 0.4040
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7093018022608082
AUC of PRC = 0.30695706169131554
min(+P, Se) = 0.32339449541284404
f1_score = nan
Epoch 0: Validation AUROC = 0.7093


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])



------------ Save best model ------------



<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

Epoch 97 Batch 0: Train Loss = 0.2660
Epoch 97 Batch 50: Train Loss = 0.2767
Epoch 97: Train Loss = 0.2770
Epoch 97: Validation Loss = 0.3134
confusion matrix:
[[2633   93]
 [ 307  125]]
accuracy = 0.8733375668525696
precision class 0 = 0.8955782055854797
precision class 1 = 0.5733944773674011
recall class 0 = 0.9658840894699097
recall class 1 = 0.28935185074806213
AUC of ROC = 0.8253070568734545
AUC of PRC = 0.46214157838428915
min(+P, Se) = 0.45622119815668205
f1_score = 0.3846153662946805
Epoch 97: Validation AUROC = 0.8253


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 0: Train Loss = 0.3264
Epoch 98 Batch 50: Train Loss = 0.2727
Epoch 98: Train Loss = 0.2734
Epoch 98: Validation Loss = 0.3241
confusion matrix:
[[2671   55]
 [ 352   80]]
accuracy = 0.8711209893226624
precision class 0 = 0.8835594058036804
precision class 1 = 0.5925925970077515
recall class 0 = 0.9798239469528198
recall class 1 = 0.18518517911434174
AUC of ROC = 0.8256781405396592
AUC of PRC = 0.463344879862744
min(+P, Se) = 0.4645308924485126
f1_score = 0.2821869368996481
Epoch 98: Validation AUROC = 0.8257


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 0: Train Loss = 0.2494
Epoch 99 Batch 50: Train Loss = 0.2739
Epoch 99: Train Loss = 0.2755
Epoch 99: Validation Loss = 0.3172
confusion matrix:
[[2656   70]
 [ 329  103]]
accuracy = 0.8736541867256165
precision class 0 = 0.8897822499275208
precision class 1 = 0.5953757166862488
recall class 0 = 0.9743213653564453
recall class 1 = 0.23842592537403107
AUC of ROC = 0.82149771745333
AUC of PRC = 0.4492815274117013
min(+P, Se) = 0.4585152838427948
f1_score = 0.3404958723330872
Epoch 99: Validation AUROC = 0.8215


### Test

In [31]:
file_name = 'model/concare_mc'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

# test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
#                                             listfile=os.path.join(data_path, 'test_listfile.csv'),
#                                             period_length=48.0)
# test_raw = utils.load_data(test_reader, discretizer, normalizer, return_names=True)
# test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

ConCare_MC(
  (grus): ModuleList(
    (0-75): 76 x GRU(1, 64, batch_first=True)
  )
  (time_attns): ModuleList(
    (0-75): 76 x TimeAwareSelfAttention()
  )
  (multi_attn): MultiHeadAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=False)
    (w_ks): Linear(in_features=64, out_features=64, bias=False)
    (w_vs): Linear(in_features=64, out_features=64, bias=False)
    (fc): Linear(in_features=64, out_features=64, bias=False)
    (attn): ScaledDotProductAttention(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (final_attn): FinalAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=True)
    (w_ks): Linear(in_features=64, out_features=64, bias=True)
    (w_vs): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (res): ResConnect(
    (lnorm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (pos_ff): PositionwiseFeedForward(
    (fc1): Li

In [32]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        output = model(batch_x)[0]

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2739  123]
 [ 252  122]]
accuracy = 0.8841161727905273
precision class 0 = 0.9157472252845764
precision class 1 = 0.4979591965675354
recall class 0 = 0.9570230841636658
recall class 1 = 0.32620319724082947
AUC of ROC = 0.8382782691883691
AUC of PRC = 0.40562530660724944
min(+P, Se) = 0.4509283819628647
f1_score = 0.3941841780453197


## ConCare_PE

In [33]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, dropout, max_len=5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

In [34]:
class ConCare_PE(nn.Module):
  def __init__(self, demo_input_dim, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.demo_embed = DemographicEmbed(self.demo_input_dim, self.hidden_dim1)
    self.gru = nn.GRU(self.input_dim, self.hidden_dim1, batch_first=True)
    self.pos_encoder = PositionalEncoding(self.d_model, self.dropout)
    self.time_attns = TimeAwareSelfAttention(self.hidden_dim1, self.hidden_dim2)
    self.multi_attn = MultiHeadAttention(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=self.dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=0.1)
    self.output_fc1 = nn.Linear(self.hidden_dim1, self.hidden_dim1)
    self.output_fc2 = nn.Linear(self.hidden_dim1, self.output_dim)
    self.dp = nn.Dropout(self.dropout)
  
  def forward(self, input, demo):
    # Demographic embedding
    demo_embedding = self.demo_embed(demo)

    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    embedding = self.gru(input)[0] * math.sqrt(self.d_model) # B*1*H
    embedding = self.pos_encoder(embedding)
    time_attn_output = self.time_attns(embedding)[0].unsqueeze(1) # B*1*H

    # Combine with demographic embedding
    demo_embedding = demo_embedding.unsqueeze(1)
    time_attn_output = torch.cat((time_attn_output, demo_embedding), 1)
    time_attn_output = self.dp(time_attn_output)

    # Get multi-head attention
    multi_attn_output, dev_loss = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc2(F.relu(self.output_fc1(value))))

    return output, dev_loss

### Training

In [24]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

In [35]:
model = ConCare_PE(demo_input_dim=12, input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [36]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare_pe'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get batch demographic data
    batch_demo = []
    for i in range(len(batch_name)):
      cur_id, cur_ep, _ = batch_name[i].split('_', 2)
      cur_idx = cur_id + '_' + cur_ep
      cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
      batch_demo.append(cur_demo)
    batch_demo = torch.stack(batch_demo).to(device)

    # Get model outputs
    output, decov_loss = model(batch_x, batch_demo)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    loss += 800 * decov_loss
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get batch demographic data
      batch_demo = []
      for i in range(len(batch_name)):
        cur_id, cur_ep, _ = batch_name[i].split('_', 2)
        cur_idx = cur_id + '_' + cur_ep
        cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
        batch_demo.append(cur_demo)
      batch_demo = torch.stack(batch_demo).to(device)

      # Get model outputs
      output, decov_loss = model(batch_x, batch_demo)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      loss += 10 * decov_loss
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 38071.6719
Epoch 0 Batch 50: Train Loss = 3104.8596
Epoch 0: Train Loss = 2837.4905
Epoch 0: Validation Loss = 1.1139
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.557770169288878
AUC of PRC = 0.17212789485902355
min(+P, Se) = 0.18518518518518517
f1_score = nan
Epoch 0: Validation AUROC = 0.5578


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])



------------ Save best model ------------

Epoch 1 Batch 0: Train Loss = 413.3376


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

/content/drive/My Drive/CS598_DL4H/utils/metrics.py:31: RuntimeWarning: invalid value encountered in double_scalars
  f1_score=2*prec1*rec1/(prec1+rec1)
<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 50: Train Loss = 0.5135
Epoch 98: Train Loss = 0.5090
Epoch 98: Validation Loss = 0.3977
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7309516045759626
AUC of PRC = 0.30795859642239154
min(+P, Se) = 0.36574074074074076
f1_score = nan
Epoch 98: Validation AUROC = 0.7310
Epoch 99 Batch 0: Train Loss = 0.4161


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])
<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 50: Train Loss = 0.5048
Epoch 99: Train Loss = 0.5080
Epoch 99: Validation Loss = 0.4063
confusion matrix:
[[2725    1]
 [ 432    0]]
accuracy = 0.8628879189491272
precision class 0 = 0.8631612062454224
precision class 1 = 0.0
recall class 0 = 0.9996331334114075
recall class 1 = 0.0
AUC of ROC = 0.7356491671421973
AUC of PRC = 0.30679570830666325
min(+P, Se) = 0.3541666666666667
f1_score = nan
Epoch 99: Validation AUROC = 0.7356


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:31: RuntimeWarning: invalid value encountered in double_scalars
  f1_score=2*prec1*rec1/(prec1+rec1)


### Test

In [37]:
file_name = 'model/concare_pe'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

# test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(data_path, 'test'),
#                                             listfile=os.path.join(data_path, 'test_listfile.csv'),
#                                             period_length=48.0)
# test_raw = utils.load_data(test_reader, discretizer, normalizer, return_names=True)
# test_dataset = Dataset(test_raw['data'][0], test_raw['data'][1], test_raw['names'])
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

ConCare_PE(
  (demo_embed): DemographicEmbed(
    (fc): Linear(in_features=12, out_features=64, bias=True)
  )
  (gru): GRU(76, 64, batch_first=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (time_attns): TimeAwareSelfAttention()
  (multi_attn): MultiHeadAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=False)
    (w_ks): Linear(in_features=64, out_features=64, bias=False)
    (w_vs): Linear(in_features=64, out_features=64, bias=False)
    (fc): Linear(in_features=64, out_features=64, bias=False)
    (attn): ScaledDotProductAttention(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (final_attn): FinalAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=True)
    (w_ks): Linear(in_features=64, out_features=64, bias=True)
    (w_vs): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (res): ResConnect(
    (lnorm): LayerNorm((64,), eps=1e-05, eleme

In [38]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)[0]

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2862    0]
 [ 374    0]]
accuracy = 0.8844252228736877
precision class 0 = 0.8844252228736877
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.7423532401334842
AUC of PRC = 0.28678337166738826
min(+P, Se) = 0.33066666666666666
f1_score = nan


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])


## ConCare without decorrelation loss

In [39]:
class MultiHeadAttentionNoDecov(nn.Module):
  def __init__(self, n_head, d_model):
    super().__init__()
    self.n_head = n_head
    self.d_model = d_model

    self.w_qs = nn.Linear(self.d_model, self.d_model, bias=False)
    self.w_ks = nn.Linear(self.d_model, self.d_model, bias=False)
    self.w_vs = nn.Linear(self.d_model, self.d_model, bias=False)
    self.fc = nn.Linear(self.d_model, self.d_model, bias=False)

    self.attn = ScaledDotProductAttention()

    # self.dropout = nn.Dropout(dropout)
    # self.layer_norm = nn.LayerNorm(d_model, eps=1e-6)

  def forward(self, q, k, v):
    b, len_q, len_k, len_v = q.size(0), q.size(1), k.size(1), v.size(1)
    q = self.w_qs(q).view(b, len_q, self.n_head, self.d_model // self.n_head)
    k = self.w_ks(k).view(b, len_k, self.n_head, self.d_model // self.n_head)
    v = self.w_vs(v).view(b, len_v, self.n_head, self.d_model // self.n_head)
    q, k, v = q.transpose(1, 2), k.transpose(1, 2), v.transpose(1, 2) # B*num_heads*d_input*d_k

    q, attn = self.attn(q, k, v)

    q = q.transpose(1, 2).contiguous().view(b, len_q, self.d_model) # B*d_input*d_model
    output_q = self.fc(q)

    return output_q

In [40]:
class ConCare_NoDecov(nn.Module):
  def __init__(self, demo_input_dim, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.demo_embed = DemographicEmbed(self.demo_input_dim, self.hidden_dim1)
    self.grus = nn.ModuleList([copy.deepcopy(nn.GRU(1, self.hidden_dim1, batch_first=True)) for _ in range(self.input_dim)])
    self.time_attns = nn.ModuleList([copy.deepcopy(TimeAwareSelfAttention(self.hidden_dim1, self.hidden_dim2)) for _ in range(self.input_dim)])
    self.multi_attn = MultiHeadAttentionNoDecov(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=self.dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=0.1)
    self.output_fc1 = nn.Linear(self.hidden_dim1, self.hidden_dim1)
    self.output_fc2 = nn.Linear(self.hidden_dim1, self.output_dim)
    self.dp = nn.Dropout(self.dropout)
  
  def forward(self, input, demo):
    # Demographic embedding
    demo_embedding = self.demo_embed(demo)

    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    record_embedding1 = self.grus[0](input[:, :, 0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
    time_attn_output = self.time_attns[0](record_embedding1)[0].unsqueeze(1) # B*1*H

    # All other records embeddings
    for i in range(1, feat_dim):
      embedding = self.grus[i](input[:, :, i].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
      attn = self.time_attns[i](embedding)[0].unsqueeze(1) # B*1*H
      time_attn_output = torch.cat((time_attn_output, attn), 1)
    
    # Combine with demographic embedding
    demo_embedding = demo_embedding.unsqueeze(1)
    time_attn_output = torch.cat((time_attn_output, demo_embedding), 1)
    time_attn_output = self.dp(time_attn_output)

    # Get multi-head attention
    multi_attn_output = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))[0]
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc2(F.relu(self.output_fc1(value))))

    return output

### Training

In [35]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic=True

In [41]:
model = ConCare_NoDecov(demo_input_dim=12, input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [42]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare_nodecov'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get batch demographic data
    batch_demo = []
    for i in range(len(batch_name)):
      cur_id, cur_ep, _ = batch_name[i].split('_', 2)
      cur_idx = cur_id + '_' + cur_ep
      cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
      batch_demo.append(cur_demo)
    batch_demo = torch.stack(batch_demo).to(device)

    # Get model outputs
    output = model(batch_x, batch_demo)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get batch demographic data
      batch_demo = []
      for i in range(len(batch_name)):
        cur_id, cur_ep, _ = batch_name[i].split('_', 2)
        cur_idx = cur_id + '_' + cur_ep
        cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
        batch_demo.append(cur_demo)
      batch_demo = torch.stack(batch_demo).to(device)

      # Get model outputs
      output = model(batch_x, batch_demo)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 0.6862
Epoch 0 Batch 50: Train Loss = 0.4609
Epoch 0: Train Loss = 0.4550
Epoch 0: Validation Loss = 0.4004
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.6381369561968424
AUC of PRC = 0.249524842608406
min(+P, Se) = 0.25968109339407747
f1_score = nan
Epoch 0: Validation AUROC = 0.6381


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])



------------ Save best model ------------



<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

Epoch 97 Batch 0: Train Loss = 0.2925
Epoch 97 Batch 50: Train Loss = 0.3236
Epoch 97: Train Loss = 0.3214
Epoch 97: Validation Loss = 0.3093
confusion matrix:
[[2703   23]
 [ 368   64]]
accuracy = 0.8761874437332153
precision class 0 = 0.8801693320274353
precision class 1 = 0.7356321811676025
recall class 0 = 0.9915627241134644
recall class 1 = 0.14814814925193787
AUC of ROC = 0.826186788402489
AUC of PRC = 0.47241315633732994
min(+P, Se) = 0.4700460829493088
f1_score = 0.24662812408003223
Epoch 97: Validation AUROC = 0.8262


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 0: Train Loss = 0.3478
Epoch 98 Batch 50: Train Loss = 0.3213
Epoch 98: Train Loss = 0.3199
Epoch 98: Validation Loss = 0.3077
confusion matrix:
[[2694   32]
 [ 358   74]]
accuracy = 0.876504123210907
precision class 0 = 0.8826998472213745
precision class 1 = 0.698113203048706
recall class 0 = 0.9882611632347107
recall class 1 = 0.17129629850387573
AUC of ROC = 0.8284124412380098
AUC of PRC = 0.4726881667729642
min(+P, Se) = 0.48148148148148145
f1_score = 0.27509293930046963
Epoch 98: Validation AUROC = 0.8284


<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 0: Train Loss = 0.2942
Epoch 99 Batch 50: Train Loss = 0.3222
Epoch 99: Train Loss = 0.3212
Epoch 99: Validation Loss = 0.3098
confusion matrix:
[[2709   17]
 [ 390   42]]
accuracy = 0.8711209893226624
precision class 0 = 0.8741529583930969
precision class 1 = 0.7118644118309021
recall class 0 = 0.9937637448310852
recall class 1 = 0.0972222238779068
AUC of ROC = 0.8298645077579413
AUC of PRC = 0.47147945837359523
min(+P, Se) = 0.4618937644341801
f1_score = 0.17107943559529498
Epoch 99: Validation AUROC = 0.8299


### Test

In [43]:
file_name = 'model/concare_nodecov'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

ConCare_NoDecov(
  (demo_embed): DemographicEmbed(
    (fc): Linear(in_features=12, out_features=64, bias=True)
  )
  (grus): ModuleList(
    (0-75): 76 x GRU(1, 64, batch_first=True)
  )
  (time_attns): ModuleList(
    (0-75): 76 x TimeAwareSelfAttention()
  )
  (multi_attn): MultiHeadAttentionNoDecov(
    (w_qs): Linear(in_features=64, out_features=64, bias=False)
    (w_ks): Linear(in_features=64, out_features=64, bias=False)
    (w_vs): Linear(in_features=64, out_features=64, bias=False)
    (fc): Linear(in_features=64, out_features=64, bias=False)
    (attn): ScaledDotProductAttention(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (final_attn): FinalAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=True)
    (w_ks): Linear(in_features=64, out_features=64, bias=True)
    (w_vs): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (res): ResConnect(
    (lnorm): LayerNorm((64,), eps=1e-05, elementwise_a

In [44]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-15-2ab15b308a62>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-21-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2836   26]
 [ 336   38]]
accuracy = 0.8881335258483887
precision class 0 = 0.8940731287002563
precision class 1 = 0.59375
recall class 0 = 0.9909154176712036
recall class 1 = 0.10160427540540695
AUC of ROC = 0.8287564883014384
AUC of PRC = 0.388782436737815
min(+P, Se) = 0.4074074074074074
f1_score = 0.17351597598329485


## Fixed time decay rate

In [40]:
class FixedDecaySelfAttention(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super().__init__()
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w_qs = nn.Parameter(torch.randn(input_dim, hidden_dim))
    self.w_ks = nn.Parameter(torch.randn(input_dim, hidden_dim))
    # self.beta = nn.Parameter(torch.zeros(1)+0.8)

    nn.init.kaiming_uniform_(self.w_qs, a=math.sqrt(5))
    nn.init.kaiming_uniform_(self.w_ks, a=math.sqrt(5))
  
  def forward(self, input):
    b, time_step, input_dim = input.size()
    time_decays = torch.tensor(range(47, -1, -1), dtype=torch.float32).unsqueeze(-1).unsqueeze(0).to(device) # 1*T*1
    b_time_decays = time_decays.repeat(b, 1, 1)+1 # B*T*1

    q = torch.matmul(input[:, -1, :], self.w_qs).reshape(b, 1, self.hidden_dim) # B*1*H
    k = torch.matmul(input, self.w_ks) # B*T*H
    v = input # B*T*H
    product = torch.matmul(q, k.transpose(1, 2)).squeeze() # B*T
    denominator = 0.8 * torch.log(math.e + (1 - F.sigmoid(product)) * (b_time_decays.squeeze()))
    weights = F.softmax(F.tanh(product / denominator)) # B*T
    output = torch.matmul(weights.unsqueeze(1), v).squeeze() # B*H

    return output, weights

In [41]:
class ConCare_FD(nn.Module):
  def __init__(self, demo_input_dim, input_dim, hidden_dim1, hidden_dim2, d_model, n_head, ff_hidden, output_dim, dropout=0.5):
    super().__init__()
    self.demo_input_dim = demo_input_dim
    self.input_dim = input_dim
    self.hidden_dim1 = hidden_dim1
    self.hidden_dim2 = hidden_dim2
    self.d_model = d_model
    self.n_head = n_head
    self.ff_hidden = ff_hidden
    self.output_dim = output_dim
    self.dropout = dropout

    self.demo_embed = DemographicEmbed(self.demo_input_dim, self.hidden_dim1)
    self.grus = nn.ModuleList([copy.deepcopy(nn.GRU(1, self.hidden_dim1, batch_first=True)) for _ in range(self.input_dim)])
    self.time_attns = nn.ModuleList([copy.deepcopy(FixedDecaySelfAttention(self.hidden_dim1, self.hidden_dim2)) for _ in range(self.input_dim)])
    self.multi_attn = MultiHeadAttention(self.n_head, self.d_model)
    self.final_attn = FinalAttention(self.hidden_dim1, self.hidden_dim1, dropout=self.dropout)
    self.res = ResConnect(self.d_model, dropout=self.dropout)
    self.pos_ff = PositionwiseFeedForward(self.d_model, self.ff_hidden, dropout=0.1)
    self.output_fc1 = nn.Linear(self.hidden_dim1, self.hidden_dim1)
    self.output_fc2 = nn.Linear(self.hidden_dim1, self.output_dim)
    self.dp = nn.Dropout(self.dropout)
  
  def forward(self, input, demo):
    # Demographic embedding
    demo_embedding = self.demo_embed(demo)

    # First record embedding
    batch_size = input.size(0)
    feat_dim = input.size(2)
    record_embedding1 = self.grus[0](input[:, :, 0].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
    time_attn_output = self.time_attns[0](record_embedding1)[0].unsqueeze(1) # B*1*H

    # All other records embeddings
    for i in range(1, feat_dim):
      embedding = self.grus[i](input[:, :, i].unsqueeze(-1), Variable(torch.zeros(batch_size, self.hidden_dim1).unsqueeze(0)).to(device))[0] # B*1*H
      attn = self.time_attns[i](embedding)[0].unsqueeze(1) # B*1*H
      time_attn_output = torch.cat((time_attn_output, attn), 1)
    
    # Combine with demographic embedding
    demo_embedding = demo_embedding.unsqueeze(1)
    time_attn_output = torch.cat((time_attn_output, demo_embedding), 1)
    time_attn_output = self.dp(time_attn_output)

    # Get multi-head attention
    multi_attn_output, dev_loss = self.res(time_attn_output, lambda x: self.multi_attn(time_attn_output, time_attn_output, time_attn_output))
    final_input = self.res(multi_attn_output, lambda x: self.pos_ff(multi_attn_output))[0]

    # Get final output
    value, score = self.final_attn(final_input)
    output = F.sigmoid(self.output_fc2(F.relu(self.output_fc1(value))))

    return output, dev_loss

### Training

In [42]:
model = ConCare_FD(demo_input_dim=12, input_dim=76, hidden_dim1=64, hidden_dim2=8, d_model=64, n_head=4, ff_hidden=256, output_dim=1).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.BCELoss()

In [43]:
max_roc = 0
max_prc = 0
train_total_loss = []
val_total_loss = []
result_history = []
file_name = 'model/concare_fd'


for epoch in range(100):
  
  # Start training
  train_batch_total_loss = []
  model.train()

  for step, (batch_x, batch_y, batch_name) in enumerate(train_loader):
    optimizer.zero_grad()
    batch_x = batch_x.float().to(device)
    batch_y = batch_y.float().to(device)

    # Get batch demographic data
    batch_demo = []
    for i in range(len(batch_name)):
      cur_id, cur_ep, _ = batch_name[i].split('_', 2)
      cur_idx = cur_id + '_' + cur_ep
      cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
      batch_demo.append(cur_demo)
    batch_demo = torch.stack(batch_demo).to(device)

    # Get model outputs
    output, decov_loss = model(batch_x, batch_demo)

    # Get loss
    loss = loss_func(output, batch_y.unsqueeze(-1))
    loss += 800 * decov_loss
    train_batch_total_loss.append(loss.cpu().detach().numpy())
    loss.backward()

    optimizer.step()

    if step % 50 == 0:
      print('Epoch %d Batch %d: Train Loss = %.4f'%(epoch, step, np.mean(np.array(train_batch_total_loss))))
    
  train_total_loss.append(np.mean(np.array(train_batch_total_loss)))
  print('Epoch %d: Train Loss = %.4f'%(epoch, np.mean(np.array(train_batch_total_loss))))

  # Start Validating
  val_batch_total_loss = []
  y_true = []
  y_pred = []

  with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(val_loader):
      batch_x = batch_x.float().to(device)
      batch_y = batch_y.float().to(device)

      # Get batch demographic data
      batch_demo = []
      for i in range(len(batch_name)):
        cur_id, cur_ep, _ = batch_name[i].split('_', 2)
        cur_idx = cur_id + '_' + cur_ep
        cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
        batch_demo.append(cur_demo)
      batch_demo = torch.stack(batch_demo).to(device)

      # Get model outputs
      output, decov_loss = model(batch_x, batch_demo)

      # Get loss
      loss = loss_func(output, batch_y.unsqueeze(-1))
      loss += 10 * decov_loss
      val_batch_total_loss.append(loss.cpu().detach().numpy())
      y_pred += list(output.cpu().detach().numpy())
      y_true += list(batch_y.cpu().numpy().flatten())
    
    val_total_loss.append(np.mean(np.array(val_batch_total_loss)))
    print('Epoch %d: Validation Loss = %.4f'%(epoch, np.mean(np.array(val_batch_total_loss))))

    y_pred = np.array(y_pred)
    y_pred = np.stack([1 - y_pred, y_pred], axis=1)
    result = metrics.print_metrics_binary(y_true, y_pred)
    result_history.append(result)

    cur_auroc = result['auroc']
    print('Epoch %d: Validation AUROC = %.4f'%(epoch, cur_auroc))
    
    if cur_auroc > max_roc:
        max_roc = cur_auroc
        state = {
            'net': model.state_dict(),
            'optimizer': optimizer.state_dict(),
            'epoch': epoch
        }
        torch.save(state, file_name)
        print('\n------------ Save best model ------------\n')

<ipython-input-40-b992205c8f48>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-20-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 0 Batch 0: Train Loss = 0.6829
Epoch 0 Batch 50: Train Loss = 0.4698
Epoch 0: Train Loss = 0.4609
Epoch 0: Validation Loss = 0.3991
confusion matrix:
[[2726    0]
 [ 432    0]]
accuracy = 0.8632045388221741
precision class 0 = 0.8632045388221741
precision class 1 = nan
recall class 0 = 1.0
recall class 1 = 0.0
AUC of ROC = 0.6890085357734844
AUC of PRC = 0.2500454733183925
min(+P, Se) = 0.25925925925925924
f1_score = nan
Epoch 0: Validation AUROC = 0.6890


/content/drive/My Drive/CS598_DL4H/utils/metrics.py:23: RuntimeWarning: invalid value encountered in float_scalars
  prec1 = cf[1][1] / (cf[1][1] + cf[0][1])



------------ Save best model ------------



<ipython-input-40-b992205c8f48>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-20-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


...

Epoch 97 Batch 0: Train Loss = 0.3133
Epoch 97 Batch 50: Train Loss = 0.2677
Epoch 97: Train Loss = 0.2627
Epoch 97: Validation Loss = 0.3142
confusion matrix:
[[2667   59]
 [ 334   98]]
accuracy = 0.8755541443824768
precision class 0 = 0.8887037634849548
precision class 1 = 0.6242038011550903
recall class 0 = 0.9783565402030945
recall class 1 = 0.22685185074806213
AUC of ROC = 0.8329104508029674
AUC of PRC = 0.4654725271311766
min(+P, Se) = 0.47453703703703703
f1_score = 0.3327674099290096
Epoch 97: Validation AUROC = 0.8329


<ipython-input-40-b992205c8f48>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-20-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 98 Batch 0: Train Loss = 0.2419
Epoch 98 Batch 50: Train Loss = 0.2676
Epoch 98: Train Loss = 0.2660
Epoch 98: Validation Loss = 0.3147
confusion matrix:
[[2663   63]
 [ 329  103]]
accuracy = 0.8758708238601685
precision class 0 = 0.8900400996208191
precision class 1 = 0.6204819083213806
recall class 0 = 0.9768891930580139
recall class 1 = 0.23842592537403107
AUC of ROC = 0.8330616015869136
AUC of PRC = 0.4701161645144195
min(+P, Se) = 0.47235023041474655
f1_score = 0.3444816077633317
Epoch 98: Validation AUROC = 0.8331


<ipython-input-40-b992205c8f48>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-20-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)


Epoch 99 Batch 0: Train Loss = 0.2634
Epoch 99 Batch 50: Train Loss = 0.2592
Epoch 99: Train Loss = 0.2598
Epoch 99: Validation Loss = 0.3128
confusion matrix:
[[2615  111]
 [ 284  148]]
accuracy = 0.8749208450317383
precision class 0 = 0.9020351767539978
precision class 1 = 0.5714285969734192
recall class 0 = 0.9592810273170471
recall class 1 = 0.34259259700775146
AUC of ROC = 0.8323041493437678
AUC of PRC = 0.46908943636424966
min(+P, Se) = 0.4793577981651376
f1_score = 0.4283646994856278
Epoch 99: Validation AUROC = 0.8323


### Test

In [44]:
file_name = 'model/concare_fd'
BATCH_SIZE = 256

checkpoint = torch.load(file_name)
save_epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['net'])
optimizer.load_state_dict(checkpoint['optimizer'])
model.eval()

ConCare_FD(
  (demo_embed): DemographicEmbed(
    (fc): Linear(in_features=12, out_features=64, bias=True)
  )
  (grus): ModuleList(
    (0-75): 76 x GRU(1, 64, batch_first=True)
  )
  (time_attns): ModuleList(
    (0-75): 76 x FixedDecaySelfAttention()
  )
  (multi_attn): MultiHeadAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=False)
    (w_ks): Linear(in_features=64, out_features=64, bias=False)
    (w_vs): Linear(in_features=64, out_features=64, bias=False)
    (fc): Linear(in_features=64, out_features=64, bias=False)
    (attn): ScaledDotProductAttention(
      (dropout): Dropout(p=0.1, inplace=False)
    )
  )
  (final_attn): FinalAttention(
    (w_qs): Linear(in_features=64, out_features=64, bias=True)
    (w_ks): Linear(in_features=64, out_features=64, bias=True)
    (w_vs): Linear(in_features=64, out_features=64, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (res): ResConnect(
    (lnorm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)

In [45]:
batch_loss = []
y_true = []
y_pred = []
with torch.no_grad():
    model.eval()
    for step, (batch_x, batch_y, batch_name) in enumerate(test_loader):
        batch_x = batch_x.float().to(device)
        batch_y = batch_y.float().to(device)
        batch_demo = []
        for i in range(len(batch_name)):
            cur_id, cur_ep, _ = batch_name[i].split('_', 2)
            cur_idx = cur_id + '_' + cur_ep
            cur_demo = torch.tensor(demographic_data[idx_list.index(cur_idx)], dtype=torch.float32)
            batch_demo.append(cur_demo)

        batch_demo = torch.stack(batch_demo).to(device)
        output = model(batch_x, batch_demo)[0]

        loss = loss_func(output, batch_y.unsqueeze(-1))
        batch_loss.append(loss.cpu().detach().numpy())
        y_pred += list(output.cpu().detach().numpy().flatten())
        y_true += list(batch_y.cpu().numpy().flatten())

print("\nTest Prediction Result")
y_pred = np.array(y_pred)
y_pred = np.stack([1 - y_pred, y_pred], axis=1)
test_res = metrics.print_metrics_binary(y_true, y_pred)

<ipython-input-40-b992205c8f48>:24: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  weights = F.softmax(F.tanh(product / denominator)) # B*T
<ipython-input-20-515625ac3faf>:26: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  score = F.softmax(attention)



Test Prediction Result
confusion matrix:
[[2812   50]
 [ 293   81]]
accuracy = 0.8940049409866333
precision class 0 = 0.9056360721588135
precision class 1 = 0.6183205842971802
recall class 0 = 0.9825296998023987
recall class 1 = 0.21657754480838776
AUC of ROC = 0.8509858107527365
AUC of PRC = 0.44539100869934967
min(+P, Se) = 0.45910290237467016
f1_score = 0.3207920865390409
